In [ ]:
from fastdebug.utils import *
from fastdebug.core import *
from fastcore.meta import *

## Reading official docs

BypassNewMeta
> BypassNewMeta (name, bases, dict)     

Metaclass: casts x to this class if it's of type cls._bypass_type

BypassNewMeta is identical to NewChkMeta, except for checking for a class as the same type, we instead check for a class of type specified in attribute _bypass_type.

In NewChkMeta, objects of the same type passed to the constructor (without arguments) would result into a new variable referencing the same object. 

However, with BypassNewMeta this only occurs if the type matches the `_bypass_type` of the class you are defining:

In [ ]:
class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=BypassNewMeta):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

In the below example, t does not refer to t2 because t is of type _TestA while _T._bypass_type is of type TestB:

In [ ]:
t = _TestA()
t2 = _T(t)
assert t is not t2

However, if t is set to _TestB to match _T._bypass_type, then both t and t2 will refer to the same object.

In [ ]:
t = _TestB()
t2 = _T(t)
t2.new_attr = 15

test_is(t, t2)
# since t2 just references t these will be the same
test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also affect t2 because they both point to the same object.
t.new_attr = 9
test_eq(t2.new_attr, 9)

In [ ]:
t = _TestB(); t
isinstance(t, _TestB)
id(_TestB)
# t2 = _T(t)
# t, t2

4536728048

## Inspecting class

In [ ]:
inspect_class(BypassNewMeta)

class BypassNewMeta(FixSigMeta):
    "Metaclass: casts `x` to this class if it's of type `cls._bypass_type`"
    def __call__(cls, x=None, *args, **kwargs):
        if hasattr(cls, '_new_meta'): x = cls._new_meta(x, *args, **kwargs)
        elif not isinstance(x,getattr(cls,'_bypass_type',object)) or len(args) or len(kwargs):
            x = super().__call__(*((x,)+args), **kwargs)
        if cls!=x.__class__: x.__class__ = cls
        return x


is BypassNewMeta a metaclass: True
is BypassNewMeta created by a metaclass: False
BypassNewMeta is created by <class 'type'>
BypassNewMeta.__new__ is object.__new__: False
BypassNewMeta.__new__ is type.__new__: False
BypassNewMeta.__new__: <function FixSigMeta.__new__>
BypassNewMeta.__init__ is object.__init__: False
BypassNewMeta.__init__ is type.__init__: True
BypassNewMeta.__init__: <slot wrapper '__init__' of 'type' objects>
BypassNewMeta.__call__ is object.__call__: False
BypassNewMeta.__call__ is type.__call__: False
BypassNewMeta.__call

## Initiating

In [ ]:
g = locals()
fdb = Fastdb(BypassNewMeta, outloc=g)
fdb.eg = """
class _TestA: pass
class _TestB: pass

print(id(_TestB))
class _T(_TestA, metaclass=BypassNewMeta):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

t = _TestA()
t2 = _T(t)
assert t is not t2
"""

fdb.eg = """
class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=BypassNewMeta):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

t = _TestB()
t2 = _T(t)
# t2.new_attr = 15

# test_is(t, t2)
# since t2 just references t these will be the same
# test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also affect t2 because they both point to the same object.
# t.new_attr = 9
# test_eq(t2.new_attr, 9)
"""

In [ ]:
fdb.snoop()

20:46:39.38 >>> Call to BypassNewMeta.__call__ in File "/tmp/BypassNewMeta.py", line 5
20:46:39.38 .......... cls = <class '__main__._T'>
20:46:39.38 .......... x = <__main__._TestB object>
20:46:39.38 .......... args = ()
20:46:39.38 .......... kwargs = {}
20:46:39.38 .......... __class__ = <class 'fastcore.meta.BypassNewMeta'>
20:46:39.38    5 |     def __call__(cls, x=None, *args, **kwargs):
20:46:39.38    6 |         if hasattr(cls, '_new_meta'): x = cls._new_meta(x, *args, **kwargs)
20:46:39.38    7 |         elif not isinstance(x,getattr(cls,'_bypass_type',object)) or len(args) or len(kwargs):
20:46:39.38    8 |             x = super().__call__(*((x,)+args), **kwargs)
20:46:39.38 .................. x = <__main__._T object>
20:46:39.38    9 |         if cls!=x.__class__: x.__class__ = cls
20:46:39.38   10 |         return x
20:46:39.38 <<< Return value from BypassNewMeta.__call__: <__main__._T object>


======================================================     Investigating BypassNewMeta     =======================================================
==============================================================     on line None     ==============================================================
     with example 
class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=BypassNewMeta):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

t = _TestB()
t2 = _T(t)
# t2.new_attr = 15

# test_is(t, t2)
# since t2 just references t these will be the same
# test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also affect t2 because they both point to the same object.
# t.new_attr = 9
# test_eq(t2.new_attr, 9)
     



In [ ]:
fdb.debug()

BypassNewMeta's dbsrc code: ==============
class BypassNewMeta(FixSigMeta):
    "Metaclass: casts `x` to this class if it's of type `cls._bypass_type`"
    import snoop
    @snoop
    def __call__(cls, x=None, *args, **kwargs):
        if hasattr(cls, '_new_meta'): x = cls._new_meta(x, *args, **kwargs)
        elif not isinstance(x,getattr(cls,'_bypass_type',object)) or len(args) or len(kwargs):
            x = super().__call__(*((x,)+args), **kwargs)
        if cls!=x.__class__: x.__class__ = cls
        return x



BypassNewMeta's example processed with dbsrc: ===============

class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=self.dbsrc):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

t = _TestB()
t2 = _T(t)
# t2.new_attr = 15

# test_is(t, t2)
# since t2 just references t these will be the same
# test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also affect t2 because they both point to the same object.
# t.new_attr = 9
# test_

In [ ]:
fdb.docsrc(3, "test", "x", "cls", "getattr(cls,'_bypass_type',object)", "id(_TestB)", "isinstance(x, _TestB)", "isinstance(x,getattr(cls,'_bypass_type',object))")

======================================================     Investigating BypassNewMeta     =======================================================
===============================================================     on line 3     ================================================================
     with example 
class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=BypassNewMeta):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

t = _TestB()
t2 = _T(t)
# t2.new_attr = 15

# test_is(t, t2)
# since t2 just references t these will be the same
# test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also affect t2 because they both point to the same object.
# t.new_attr = 9
# test_eq(t2.new_attr, 9)
     

print selected srcline with expands below--------
    "Metaclass: casts `x` to this class if it's of type `cls._bypass_type`"                                                                             (1)
    def __call__(cls, x=None, *args, *

In [ ]:
fdb.snoop(['cls._bypass_type', "isinstance(x,getattr(cls,'_bypass_type',object))"])

20:46:39.40 >>> Call to BypassNewMeta.__call__ in File "/tmp/BypassNewMeta.py", line 5
20:46:39.40 .......... cls = <class '__main__._T'>
20:46:39.40 .......... x = <__main__._TestB object>
20:46:39.40 .......... args = ()
20:46:39.40 .......... kwargs = {}
20:46:39.40 .......... __class__ = <class 'fastcore.meta.BypassNewMeta'>
20:46:39.40 .......... cls._bypass_type = <class '__main__._TestB'>
20:46:39.40 .......... isinstance(x,getattr(cls,'_bypass_type',object)) = False
20:46:39.40    5 |     def __call__(cls, x=None, *args, **kwargs):
20:46:39.40    6 |         if hasattr(cls, '_new_meta'): x = cls._new_meta(x, *args, **kwargs)
20:46:39.40    7 |         elif not isinstance(x,getattr(cls,'_bypass_type',object)) or len(args) or len(kwargs):
20:46:39.40    8 |             x = super().__call__(*((x,)+args), **kwargs)
20:46:39.40 .................. x = <__main__._T object>
20:46:39.40    9 |         if cls!=x.__class__: x.__class__ = cls
20:46:39.40   10 |         return x
20:46:39.40

======================================================     Investigating BypassNewMeta     =======================================================
==============================================================     on line None     ==============================================================
     with example 
class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=BypassNewMeta):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

t = _TestB()
t2 = _T(t)
# t2.new_attr = 15

# test_is(t, t2)
# since t2 just references t these will be the same
# test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also affect t2 because they both point to the same object.
# t.new_attr = 9
# test_eq(t2.new_attr, 9)
     



In [ ]:
fdb.debug()

BypassNewMeta's dbsrc code: ==============
class BypassNewMeta(FixSigMeta):
    "Metaclass: casts `x` to this class if it's of type `cls._bypass_type`"
    import snoop
    @snoop(watch=("cls._bypass_type","isinstance(x,getattr(cls,'_bypass_type',object))"))
    def __call__(cls, x=None, *args, **kwargs):
        if hasattr(cls, '_new_meta'): x = cls._new_meta(x, *args, **kwargs)
        elif not isinstance(x,getattr(cls,'_bypass_type',object)) or len(args) or len(kwargs):
            x = super().__call__(*((x,)+args), **kwargs)
        if cls!=x.__class__: x.__class__ = cls
        return x



BypassNewMeta's example processed with dbsrc: ===============

class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=self.dbsrc):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

t = _TestB()
t2 = _T(t)
# t2.new_attr = 15

# test_is(t, t2)
# since t2 just references t these will be the same
# test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also